In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

#import xarray as xr

from shapely.wkt import loads
from shapely.geometry import Polygon
from shapely.geometry import Point

import rasterio
import rasterio.sample

from functools import partial
from scipy.spatial.distance import cdist

In [2]:
parquet_files = {'marine_protected_areas': ('../data/PR100/Habitat/marine_protected_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'land_nationwide_hapc': ('../data/PR100/Habitat/land_nationwide_hapc.parquet',
  'EPSG:32161',
  'centroid'),
 'artificial_reefs': ('../data/PR100/Habitat/artificial_reefs.parquet',
  'EPSG:32161',
  'irrelevant'),
 'nationwide_hapc': ('../data/PR100/Habitat/nationwide_hapc.parquet',
  'EPSG:32161',
  'irrelevant'),
 'areas_protegidas_agencia': ('../data/PR100/Habitat/areas_protegidas_agencia.parquet',
  'EPSG:32161',
  'centroid'),
 'noaa_charted_submarine_cables_30m_buffer': ('../data/PR100/Infrastructure/noaa_charted_submarine_cables_30m_buffer.parquet',
  'EPSG:32161',
  'irrelevant'),
 'setbacks_xmission': ('../data/PR100/Infrastructure/setbacks_xmission.parquet',
  'EPSG:32161',
  'lines'),
 'submarine_cable_30m_buffer': ('../data/PR100/Infrastructure/submarine_cable_30m_buffer.parquet',
  'EPSG:32161',
  'irrelevant'),
 'pipeline_areas': ('../data/PR100/Infrastructure/pipeline_areas.parquet',
  'EPSG:32161',
  'irrelevant'),
 'setbacks_roads': ('../data/PR100/Infrastructure/setbacks_roads.parquet',
  'EPSG:4326',
  'density'),
 'setbacks_runway': ('../data/PR100/Infrastructure/setbacks_runway.parquet',
  'EPSG:4326',
  'centroid'),
 'setbacks_airport': ('../data/PR100/Infrastructure/setbacks_airport.parquet',
  'EPSG:4326',
  'points'),
 'setbacks_buildings': ('../data/PR100/Infrastructure/setbacks_buildings.parquet',
  'EPSG:4326',
  'density'),
 'federal_and_state_waters': ('../data/PR100/Boundaries/federal_and_state_waters.parquet',
  'EPSG:32161',
  'centroid'),
 'coastal_zone_mgmt_act_boundary': ('../data/PR100/Boundaries/coastal_zone_mgmt_act_boundary.parquet',
  'EPSG:32161',
  'centroid'),
 'land_protected_areas': ('../data/PR100/Boundaries/land_protected_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'water_bodies': ('../data/PR100/Topography/water_bodies.parquet',
  'EPSG:4326',
  'centroid'),
 'wrecks_and_obstructions': ('../data/PR100/Hazards/wrecks_and_obstructions.parquet',
  'EPSG:4326',
  'points'),
 'ocean_disposal_sites': ('../data/PR100/Hazards/ocean_disposal_sites.parquet',
  'EPSG:32161',
  'centroid'),
 'tropical_cyclone_storm_segments': ('../data/PR100/Hazards/tropical_cyclone_storm_segments.parquet',
  'EPSG:32161',
  'lines'),
 'tropical_cyclone_wind_exposure': ('../data/PR100/Hazards/tropical_cyclone_wind_exposure.parquet',
  'EPSG:32161',
  'wind_exposure'),
 'unexploded_ordinance_areas_fuds': ('../data/PR100/Hazards/unexploded_ordinance_areas_fuds.parquet',
  'EPSG:32161',
  'irrelevant'),
 'danger_zones_and_restricted_areas': ('../data/PR100/Hazards/danger_zones_and_restricted_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'unexploded_ordinance_areas': ('../data/PR100/Hazards/unexploded_ordinance_areas.parquet',
  'EPSG:4326',
  'centroid')}

In [3]:
tif_files = ['../data/wind_atlas/PRI_power-density_50m.tif',
 '../data/wind_atlas/PRI_power-density_100m.tif',
 '../data/wind_atlas/PRI_power-density_150m.tif',
 '../data/wind_atlas/PRI_power-density_200m.tif',
 '../data/wind_atlas/PRI_power-density_10m.tif',
 '../data/wind_atlas/PRI_wind-speed_100m.tif',
 '../data/solar_atlas/Yearly_PVOUT.tif',
 '../data/solar_atlas/Daily_PVOUT.tif',
 '../data/PR100/Topography/slope.tif',
 '../data/PR100/Topography/elevation.tif']

In [4]:
binary_mask_columns = ['marine_protected_areas', 'land_nationwide_hapc',
       'areas_protegidas_agencia', 'setbacks_xmission',
       'setbacks_runway', 'setbacks_airport',
       'federal_and_state_waters', 'coastal_zone_mgmt_act_boundary',
       'land_protected_areas', 'water_bodies', 'wrecks_and_obstructions',
       'ocean_disposal_sites', 'danger_zones_and_restricted_areas',
       'unexploded_ordinance_areas']

In [5]:
def find_parquet_files(directory):
    """
    Find all Parquet files within a directory and its subdirectories.

    :param directory: The base directory to search.
    :return: A list of paths to Parquet files.

    Note: This function isn't used in this notebook, because the file list
    has already been generated and enriched (see 'parquet_files' dict above)
    """
    parquet_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.parquet'):
                parquet_files.append(os.path.join(root, file))
    return parquet_files

In [6]:
def find_tif_files(directory):
    """
    Find all TIF files within a directory and its subdirectories.

    :param directory: The base directory to search.
    :return: A list of paths to TIF files.

    Note: This function isn't used in this notebook, because the file list
    has already been generated (see 'tif_files' dict above)
    """
    tif_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tif'):
                tif_files.append(os.path.join(root, file))
    return tif_files

In [7]:
def get_gdf(parquet_file, crs, grid):
    df = pd.read_parquet(parquet_file)
    df['geometry'] = df['geometry'].apply(loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.set_crs(crs, inplace=True)
    gdf.to_crs(grid.crs, inplace=True)
    return gdf

In [8]:
def create_grid(x_start, x_end, y_start, y_end, square_size):
    grid = []
    for x in np.arange(x_start, x_end, square_size):
        for y in np.arange(y_start, y_end, square_size):
            # Define the square's coordinates
            square = Polygon([(x, y), (x + square_size, y), (x + square_size, y + square_size), (x, y + square_size)])
            grid.append(square)
    return gpd.GeoDataFrame({'geometry': grid}, crs="EPSG:32161")

In [9]:
# This is a legacy function and is no longer in use

def calculate_cyclone_risk_score(df):
    # Normalize the wind speed occurrences
    max_occurrence = df[['occurrence34kt', 'occurrence50kt', 'occurrence64kt']].max().max()
    df['norm_occurrence34kt'] = df['occurrence34kt'] / max_occurrence
    df['norm_occurrence50kt'] = df['occurrence50kt'] / max_occurrence
    df['norm_occurrence64kt'] = df['occurrence64kt'] / max_occurrence

    # Normalize major hurricanes occurrences
    max_major_hurricanes = df['occurrence64ktMajorHurricanes'].max()
    df['norm_major_hurricanes'] = df['occurrence64ktMajorHurricanes'] / max_major_hurricanes

    # Normalize intersecting storm tracks
    max_intersecting = df['intersectingStormTracks'].max()
    df['norm_intersectingTracks'] = df['intersectingStormTracks'] / max_intersecting

    # Assign weights and calculate risk score
    weights = {
        'norm_occurrence34kt': 0.1,
        'norm_occurrence50kt': 0.2,
        'norm_occurrence64kt': 0.3,
        'norm_major_hurricanes': 0.3,
        'norm_intersectingTracks': 0.1
    }

    df['cyclone_risk_score'] = sum(df[col] * weight for col, weight in weights.items())
    
    return df

In [10]:
def calculate_cyclone_failure_percentage(df):
    probability_of_hurricane_occuring = df['occurrence64ktMajorHurricanes'] / 117
    probability_of_failure_due_to_hurricane = 0.5
    average_life_span = 25
    df['turbine_cyclone_failure_percentage'] = probability_of_hurricane_occuring * probability_of_failure_due_to_hurricane * average_life_span / 2
    return df

In [11]:
def rasterize_cyclone_failure_risk(grid_gdf, cyclone_data_gdf):
    # Ensure both dataframes are in the same CRS
    cyclone_data_gdf = cyclone_data_gdf.to_crs(grid_gdf.crs)

    # Spatial join - to associate cyclone data with each grid cell
    joined_gdf = gpd.sjoin(grid_gdf, cyclone_data_gdf, how='left', predicate='intersects')

    # Apply the failure cost calculation to each row
    joined_gdf = calculate_cyclone_failure_percentage(joined_gdf)

    # Group by grid cell index and average the scores within each cell
    aggregated_scores = joined_gdf.groupby(joined_gdf.index)['turbine_cyclone_failure_percentage'].mean()

    return aggregated_scores.reindex(grid_gdf.index, fill_value=0)

In [12]:
def rasterize(grid, data, type):
    joined_gdf = gpd.sjoin(grid, data[['geometry']], how='right', predicate='intersects')
    count = joined_gdf.groupby('index_left').size().reindex(grid.index, fill_value=0)
    if type == 'density':
        return count
    else:
        return count > 0

In [13]:
def process_parquet_file(file_path, crs, type, handle, grid):
    gdf = get_gdf(file_path, crs, grid)
    if type == 'irrelevant':
        pass
    elif type == 'wind_exposure':
        grid[handle] = rasterize_cyclone_failure_risk(grid, gdf)
    else:
        grid[handle] = rasterize(grid, gdf, type)

In [14]:
def process_parquet_files(files, grid):
    for handle, file_data in files.items():
        file_path = file_data[0]
        crs = file_data[1]
        type = file_data[2]
        print(f'Processing file: {file_path}')
        process_parquet_file(file_path, crs, type, handle, grid)

In [15]:
def process_tif_file(file_path, grid, coords):
    handle = file_path.split('/')[-1].split('.')[0]
    # Open the raster and sample values at each coordinate
    with rasterio.open(file_path) as src:
        if grid.crs != src.crs:
            centroids = grid.geometry.centroid.to_crs(src.crs)
            coords = [(x,y) for x, y in zip(centroids.geometry.x, centroids.geometry.y)]

        # Sample the raster at each centroid location
        values = [x[0] for x in rasterio.sample.sample_gen(src, coords)]
    
    # Assign the values to the grid dataframe
    grid[handle] = values

In [16]:
def process_hourly_file(grid, hourly):
    # Ensure both DataFrames are in the same CRS
    if grid.crs != hourly.crs:
        hourly_wind = hourly.to_crs(grid.crs)

    # Iterate over the columns in hourly and assign values to grid
    for column in hourly.columns:
        joined = gpd.sjoin_nearest(grid, hourly[[column, 'geometry']], how='left')
        grid[column] = joined[column]

    return grid

In [17]:
def process_tif_files(files, grid):
    # Compute centroids
    centroids = grid.geometry.centroid
    # Create an array of coordinates in (x, y) format
    coords = [(x,y) for x, y in zip(centroids.geometry.x, centroids.geometry.y)]

    for file_path in tif_files:
        print(f'Processing file: {file_path}')
        process_tif_file(file_path, grid, coords)

In [18]:
def create_final_binary_mask(grid, binary_mask_columns):
    """
    Create a final binary mask column in the grid dataframe, which is True if any of 
    the specified binary mask columns are True.

    :param grid: DataFrame containing the grid information.
    :param binary_mask_columns: List of columns to be considered for the binary mask.
    :return: Updated DataFrame with an additional 'final_binary_mask' column.
    """

    # Check if any of the specified columns are True for each row
    grid['masked'] = grid[binary_mask_columns].any(axis=1)

In [19]:
def plot_columns(gdf):
    for column in gdf.columns:
        if column in ['geometry', 'centroid']:
            continue
        if not 'wind' in column and not 'PRI' in column:
            continue
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        environment_gdf.plot(column=column, ax=ax, legend=True)
        plt.title(column)
        plt.show()

In [20]:
environment_gdf = create_grid(100000, 300000, 200000, 300000, 500)

In [21]:
process_parquet_files(parquet_files, environment_gdf)

Processing file: ../data/PR100/Habitat/marine_protected_areas.parquet
Processing file: ../data/PR100/Habitat/land_nationwide_hapc.parquet
Processing file: ../data/PR100/Habitat/artificial_reefs.parquet
Processing file: ../data/PR100/Habitat/nationwide_hapc.parquet
Processing file: ../data/PR100/Habitat/areas_protegidas_agencia.parquet
Processing file: ../data/PR100/Infrastructure/noaa_charted_submarine_cables_30m_buffer.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_xmission.parquet
Processing file: ../data/PR100/Infrastructure/submarine_cable_30m_buffer.parquet
Processing file: ../data/PR100/Infrastructure/pipeline_areas.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_roads.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_runway.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_airport.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_buildings.parquet
Processing file: ../data/PR100/Boundaries/federal_and_state_wat

In [22]:
process_tif_files(tif_files, environment_gdf)

Processing file: ../data/wind_atlas/PRI_power-density_50m.tif
Processing file: ../data/wind_atlas/PRI_power-density_100m.tif
Processing file: ../data/wind_atlas/PRI_power-density_150m.tif
Processing file: ../data/wind_atlas/PRI_power-density_200m.tif
Processing file: ../data/wind_atlas/PRI_power-density_10m.tif
Processing file: ../data/wind_atlas/PRI_wind-speed_100m.tif
Processing file: ../data/solar_atlas/Yearly_PVOUT.tif
Processing file: ../data/solar_atlas/Daily_PVOUT.tif
Processing file: ../data/PR100/Topography/slope.tif
Processing file: ../data/PR100/Topography/elevation.tif


In [23]:
create_final_binary_mask(environment_gdf, binary_mask_columns)

In [24]:
environment_gdf['centroid'] = environment_gdf.centroid

In [25]:
environment_gdf['occupied'] = False

## Calculating per-cell energy demand

In [26]:
total_building_density = environment_gdf['setbacks_buildings'].sum()

total_power_density = 100000 # TODO -- Get total power demand for PR

total_power_density_per_building_density = total_power_density / total_building_density

environment_gdf['demand'] = environment_gdf['setbacks_buildings'] * total_power_density_per_building_density

## Calculating dist to nearest transmission line

In [29]:
def nearest_transmission_distance(row, gdf):
    if row['setbacks_xmission'] or row['masked']:
        return 0
    else:
        # Calculate distances to all cells where setbacks_xmission is True
        distances = gdf[gdf['setbacks_xmission']]['geometry'].distance(row['geometry'])
        # Return the minimum distance
        return distances.min()

In [30]:
environment_gdf['distance_to_transmission_line'] = environment_gdf.apply(nearest_transmission_distance, gdf=environment_gdf, axis=1)

# Incorporating ERA5 Wind Speed Data

In [ ]:
ds = xr.open_dataset("../data/generation_and_demand/2019_wind_speeds.nc")
ds

In [ ]:
total_wind_speed_array = ds.total_wind_speed
total_wind_speed_np = np.array(total_wind_speed_array)

print(total_wind_speed_np.shape)

total_wind_speed_array

In [ ]:
hrs = np.array(ds.coords['hour'])
lats = np.array(ds.coords['latitude'])
lons = np.array(ds.coords['longitude'])

lists = []
for lat_idx in range(0, ds.coords['latitude'].shape[0]):
    for lon_idx in range(0, ds.coords['longitude'].shape[0]):
        hour_wind_speeds = {}
    
        this_point_wind_speeds = total_wind_speed_np[:, lat_idx, lon_idx]
        this_normalized_wind_speeds = this_point_wind_speeds/np.mean(this_point_wind_speeds)
        
        for hr_idx in range(0, ds.coords['hour'].shape[0]):
            this_point = Point(lats[lat_idx], lons[lon_idx])
    
            this_hour = hrs[hr_idx]

            hour_wind_speeds[f'normalized_wind_speed_hour_{this_hour}'] = this_normalized_wind_speeds[hr_idx]
            hour_wind_speeds[f'wind_speed_hour_{this_hour}'] = this_point_wind_speeds[hr_idx]
        

        lists.append({'latitude': lats[lat_idx], 'longitude': lons[lon_idx],'geometry': this_point, **hour_wind_speeds})

diurnal_wind_speeds = gpd.GeoDataFrame(lists)
diurnal_wind_speeds

### Kevin

Hey, so I need to do something here. I know how to do this in geopandas, but I think our `environment_gdf` has points defined in a non lat-lon reference system? So I'm not totally sure how to do this. 

Basically I want to merge the columns beginning with `normalized_wind_speed_hour_` in the `diurnal_wind_speeds` dataframe above onto our `environment_gdf`. Then, I can multiple each of these columns by `PRI_wind-speed_100m` as seen below. This will give us the diurnal mean wind speed values (using the wind speed curves from ERA5 but the mean wind speed values from Global Wind Atlas TIFs). 

Finally, I'll apply `power_curve_function` as seen below to get actual power values. 

Does this make sense? Is this something you could help with? 

In [ ]:
environment_gdf['PRI_wind-speed_100m']

In [ ]:
power_curve = {
    3: 25,
    3.5: 89, 
    4: 171, 
    4.5: 269, 
    5: 389,
    5.5: 533,
    6: 704,
    6.5: 906,
    7: 1136,
    7.5: 1400,
    8: 1674,
    8.5: 1934,
    9: 2160,
    9.5: 2316,
    10: 2416, 
    10.5: 2477,
    11: 2514,
    12: 2530, 
    12.5: 2530, 
    25: 2530,
}

power_curve_key = np.array(list(power_curve.keys()))
power_curve_val = np.array(list(power_curve.values()))

power_curve_function = partial(np.interp, xp=power_curve_key, fp=power_curve_val, left=0, right=0)

power_curve_function(3.8)

In [ ]:
raw_wind_speed_cols = [col for col in diurnal_wind_speeds.columns.values if col[0:11] == 'wind_speed_']
power_cols = [col.replace('wind_speed', 'power_kw') for col in raw_wind_speed_cols]

diurnal_wind_speeds[power_cols] = diurnal_wind_speeds[raw_wind_speed_cols].map(power_curve_function)

In [ ]:
diurnal_wind_speeds.iloc[0][power_cols].plot()

In [ ]:
environment_gdf.to_parquet('../data/processed/environment_gdf.parquet')

In [ ]:
plot_columns(environment_gdf)

# Calculating Power Output

In [ ]:
def calculate_daily_power_output(environment_gdf):
    solar_columns = [[f'solar_power_{i}' for i in range(1, 25)]]
    wind_columns = [[f'solar_power_{i}' for i in range(1, 25)]]
    environment_gdf['daily_solar_output'] = solar_columns.sum(axis=1)
    environment_gdf['daily_wind_output'] = wind_columns.sum(axis=1)

In [ ]:
def calculate_power_output_reward(environment_gdf, demand):
    """Uses supply and demand curve to determine the amount of demand satisfied by
    the solar and wind installations"""
    cost_kWh = 9999  # TODO

    # Filter for solar and wind installations
    solar_gdf = environment_gdf[(environment_gdf['occupied']) & (environment_gdf['installation'] == 'solar')]
    wind_gdf = environment_gdf[(environment_gdf['occupied']) & (environment_gdf['installation'] == 'wind')]

    # Prepare column names for solar and wind power
    solar_power_columns = [f'solar_power_{i}' for i in range(1, 25)]
    wind_power_columns = [f'wind_power_{i}' for i in range(1, 25)]

    # Vectorized sum of power output for solar and wind for each hour
    total_solar_power = solar_gdf[solar_power_columns].sum()
    total_wind_power = wind_gdf[wind_power_columns].sum()

    # Calculate the reward using vectorized minimum
    total_power = total_solar_power + total_wind_power
    reward = np.minimum(total_power, demand[1:]).sum() * cost_kWh

    return reward

# Calculating Installation Costs

In [ ]:
def calculate_installation_cost(gdf):
    solar_install_cost_per_cell = 9999 #TODO
    wind_install_cost_per_cell = 9999 #TODO

    solar_cost = gdf[gdf.installation == 'solar'] * 1.05 * solar_install_cost_per_cell
    wind_cost = gdf[gdf.installation == 'wind'] * 1.05 * wind_install_cost_per_cell